In [47]:
import pandas as pd

In [48]:
# load the plots_and_genres.csv file into a dataframe
df = pd.read_csv('./data/plots_and_genres.csv')
df

,movie,plot,genres
0,"""#7DaysLater"" (2013)",#7dayslater is an interactive comedy series fe...,['Comedy']
1,"""#Cake"" (2015)",#CAKE is a hour-long serial narrative comedy a...,['Comedy']
2,"""#DaddyLeaks"" (????)",The life of four close friends in their late t...,['Comedy']
3,"""#Elmira"" (2014)",#Elmira follows the story of a bunch of strang...,['Comedy']
4,"""#Fuga"" (2016)","Months after an apocalyptic event, a group of ...","['Action', 'Drama', 'Horror']"
...,...,...,...
347530,� propos: philosophie (2016),Is happiness good for human beings? Is truth m...,['Documentary']
347531,�a go�te le ciel (2014),The 'Sky's the Limit' is the story of an 11 ye...,"['Drama', 'Family', 'Fantasy', 'Short']"
347532,� solo un nastro che gira (2017),A woman in crisis finds in a peculiar bar the ...,"['Comedy', 'Drama', 'Romance', 'Short']"
347533,�X-Driver the Movie (2002) (V),"The squeal of smoking tires, the roar of the e...","['Action', 'Animation']"


In [49]:
# count nan values
df.isna().sum()

movie     0
plot      0
genres    0
dtype: int64

In [50]:

genres_to_consider = [
    'Drama',
    'Comedy',
    'Thriller',
    'Romance',
    'Action',
    'Family',
    'Horror',
    'Crime',
    'Adventure',
    'Animation',
    'Fantasy',
    'Sci-Fi',
    'Mystery',
    'Biography',
    'Music',
    'History',
    'War',
    'Western',
    'Sport',
    'Musical',
]

In [56]:
# filter the dataframe to only include the genres we want to consider

def check(row):
    sublist = row['genres']
    # remove quotes from each item in the list
    sublist = [item.strip().strip("'") for item in sublist.strip('][').split(',')]

    # # if even one of the genres in the list is in the genres_to_consider list, return True
    # for genre in sublist:
    #     if genre in genres_to_consider:
    #         return True
    
    # check intersection of two lists
    if len(set(sublist).intersection(set(genres_to_consider))) > 0:
        # set genres to the intersection of the two lists
        row['genres'] = list(set(sublist).intersection(set(genres_to_consider)))
        return True
    
    return False

In [57]:

filtered_df = df[df.apply(check, axis=1)]

In [59]:
filtered_df['genres'].value_counts()

genres
[Drama]                                                  57939
[Comedy]                                                 34150
[Drama, Comedy]                                           8249
[Drama, Romance]                                          7044
[Horror]                                                  6740
                                                         ...  
[Animation, Horror, Music, Sci-Fi, Adventure, Action]        1
[Horror, History, Drama, Fantasy, Action]                    1
[Western, Crime, Romance, Comedy]                            1
[Biography, Adventure, War, Action]                          1
[Family, History, Drama, War, Action]                        1
Name: count, Length: 4924, dtype: int64